In [3]:
import pandas as pd
import os
import numpy as np
from scipy.stats import spearmanr

In [5]:
def compute_rho_eb_te_cr(data_name):
    cr_df = pd.read_csv('../results/cr/all_cr.csv')
    cr_df = cr_df[cr_df.data == data_name]
    te_df = pd.read_csv('../results/te/transformation_error.csv')
    te_df = te_df[te_df.data == data_name]
    te_df = te_df[te_df.metric == 'nrmse']
    cr_df = cr_df.rename({'error_bound': 'eb', 'gzip': 'cr'}, axis=1)
    te_df = te_df.rename({'decompression error': 'te'}, axis=1)
    cr_df = cr_df[cr_df.eb.isin(te_df.eb)]
    cr_df.set_index(['eb', 'compression'], inplace=True)
    te_df.set_index(['eb', 'compression'], inplace=True)
    cr_df.drop(['data'], axis=1, inplace=True)
    te_df.drop(['data'], axis=1, inplace=True)
    joined = cr_df.join(te_df)
    joined.reset_index(inplace=True)
    display(joined.groupby('compression').corr())
    return joined

def compute_rho_tfe_te_cr(data_name):
    all_models_results = pd.read_csv('../results/tfe/all_models_results.csv')
    all_models_results = all_models_results[all_models_results.data == data_name]
    all_models_results = all_models_results[all_models_results.te_metric == 'nrmse']
    if 'SOLAR' in data_name:
        all_models_results = all_models_results[all_models_results.model != 'GRU']
    all_models_results.drop(['forecasting error', 'error'], axis=1, inplace=True)
    display(all_models_results.groupby('compression').corr(method='spearman'))
    # columns = all_models_results.columns.tolist()
    # ['compression', 'data', 'eb', 'tfe_metric', 'model', 'compression ratio', 'TFE', 'te_metric', 'te']
    columns = ['compression ratio', 'TFE', 'te']

    all_corr_results = pd.DataFrame()

    for eblc in ['PMC', 'SWING', 'SZ']:
        correlations = {}
        pvalues = {}
        ebcl_results = all_models_results[all_models_results.compression == eblc]
        for col1 in range(3):
            for col2 in range(col1, 3):
                if col1 != col2:
                    corr, pvalue = spearmanr(ebcl_results[columns[col1]], ebcl_results[columns[col2]])
                    correlations[f"{columns[col1]}_{columns[col2]}"] = corr
                    pvalues[f"{columns[col1]}_{columns[col2]}"] = pvalue
        eblc_corr = pd.DataFrame([correlations, pvalues], index=['correlation', 'pvalue'])
        eblc_corr['eblc'] = eblc
        all_corr_results = pd.concat([all_corr_results, eblc_corr])
    return all_corr_results

def compute_rho_tfe_te_cr_pweather():
    pweather_results = pd.read_csv('../results/tfe/pprocessed_weather.csv')
    pweather_results.drop(['forecasting error'], axis=1, inplace=True)
    display(pweather_results.groupby('compression').corr(method='spearman'))

def get_tfe(model_results):
    baseline_result  = model_results[model_results.eblc == 'baseline']
    model_results['tfe'] = (model_results.nrmse.values - baseline_result.nrmse.values)/baseline_result.nrmse.values
    baseline_result = baseline_result.append([baseline_result]*2, ignore_index=True)
    baseline_result.at[0, 'eblc'] = 'pmc'
    baseline_result.at[1, 'eblc'] = 'sz'
    baseline_result.at[2, 'eblc'] = 'swing'
    baseline_result['tfe'] = 0.
    return pd.concat([model_results, baseline_result])

def compute_rho_tfe_te_cr_aus():
    all_results = pd.DataFrame()
    for root, dirs, files in os.walk('../results/tfe/per_model/'):
        for result_file in files:
            if 'gru' in result_file or 'arima' in result_file:
                continue
            print(result_file)
            model_results = pd.read_csv(os.path.join(root, result_file))
            model_results = model_results[(model_results.data == 'aus')]
            model_results = get_tfe(model_results)
            all_results = pd.concat([all_results, model_results])

    all_results = all_results[['error', 'tfe', 'eblc']]

    return all_results


def get_feature_diff(features):
    raw_feat = features.iloc[0]
    num_colums = features.columns[:-3]
    features[num_colums] = features[num_colums].astype(float)
    features[num_colums] -= raw_feat[num_colums].astype(float)
    features[num_colums] = features[num_colums].astype(float)
    raw_feat = features[features.compression == 'RAW']
    raw_feat = raw_feat.append([raw_feat], ignore_index=True)
    features.at[0, 'compression'] = 'PMC'
    raw_feat.at[0, 'compression'] = 'SZ'
    raw_feat.at[1, 'compression'] = 'SWING'
    return pd.concat([raw_feat, features])


def get_feature_diff_percentage(features):
    raw_feat = features.iloc[0]
    num_colums = features.columns[:-3]
    features[num_colums] = features[num_colums].astype(float)
    features[num_colums] = (features[num_colums] - raw_feat[num_colums].astype(float))/raw_feat[num_colums].astype(float)
    features[num_colums] = features[num_colums].astype(float)
    raw_feat = features[features.compression == 'RAW']
    raw_feat = raw_feat.append([raw_feat], ignore_index=True)
    features.at[0, 'compression'] = 'PMC'
    raw_feat.at[0, 'compression'] = 'SZ'
    raw_feat.at[1, 'compression'] = 'SWING'
    return pd.concat([raw_feat, features])

def get_features_diff_solar(features):
    raw_feat = features.iloc[0]
    num_colums = features.columns[2:-1]
    features[num_colums] = features[num_colums].astype(float)
    features[num_colums] -= raw_feat[num_colums].astype(float)
    features[num_colums] = features[num_colums].astype(float)
    raw_feat = features[features.compression == 'RAW']
    raw_feat = raw_feat.append([raw_feat], ignore_index=True)
    features.at[0, 'compression'] = 'PMC'
    raw_feat.at[0, 'compression'] = 'SZ'
    raw_feat.at[1, 'compression'] = 'SWING'
    return pd.concat([raw_feat, features])

def get_features_diff_percentage_solar(features):
    raw_feat = features.iloc[0]
    num_colums = features.columns[2:-1]
    features[num_colums] = features[num_colums].astype(float)
    features[num_colums] = (features[num_colums] - raw_feat[num_colums].astype(float))/raw_feat[num_colums].astype(float)
    features[num_colums] = features[num_colums].astype(float)
    raw_feat = features[features.compression == 'RAW']
    raw_feat = raw_feat.append([raw_feat], ignore_index=True)
    features.at[0, 'compression'] = 'PMC'
    raw_feat.at[0, 'compression'] = 'SZ'
    raw_feat.at[1, 'compression'] = 'SWING'
    return pd.concat([raw_feat, features])

def get_baselines_well(features):
    num_colums = features.columns[:-3]
    features[num_colums] = features[num_colums].astype(float)
    raw_feat = features[features.compression == 'RAW']
    raw_feat = raw_feat.append([raw_feat], ignore_index=True)
    features.at[0, 'compression'] = 'PMC'
    raw_feat.at[0, 'compression'] = 'SZ'
    raw_feat.at[1, 'compression'] = 'SWING'
    return pd.concat([raw_feat, features])

def get_baselines_well_solar(features):
    num_colums = features.columns[2:-1]
    features[num_colums] = features[num_colums].astype(float)
    raw_feat = features[features.compression == 'RAW']
    raw_feat = raw_feat.append([raw_feat], ignore_index=True)
    features.at[0, 'compression'] = 'PMC'
    raw_feat.at[0, 'compression'] = 'SZ'
    raw_feat.at[1, 'compression'] = 'SWING'
    return pd.concat([raw_feat, features])


def compute_features():
    all_models_results = pd.read_csv('../results/tfe/models_results.csv')
    datasets = all_models_results.data.str.lower().unique()
    all_datasets_features = pd.DataFrame()
    for dataset in datasets:
        print(dataset)
        features = pd.read_csv(f'../results/features/test_{dataset}_features.csv')
        x = features.eb.values
        features.eb = np.where(x >= 1.0, x*0.01, x)
        features.eblc = features.eblc.str.upper()
        features.rename({'eblc': 'compression'}, axis=1, inplace=True)
        features = get_baselines_well(features) if 'solar' not in dataset else get_baselines_well_solar(features)
        features.set_index(['compression', 'eb'], inplace=True)
        model_results = all_models_results[all_models_results.data == dataset.upper()]
        model_results = model_results[model_results.te_metric == 'nrmse']
        model_results = model_results[['eb',  'te', 'TFE', 'compression']]
        joined_df = features.merge(model_results.reset_index(drop=True), on=['compression', 'eb']).drop_duplicates()
        joined_df['data'] = dataset
        all_datasets_features = pd.concat([all_datasets_features, joined_df])

    all_datasets_features.to_csv('../results/features/all_datasets_features.csv', index=False)


def compute_features_diff_percentage():
    all_models_results = pd.read_csv('../results/tfe/models_results.csv')
    datasets = all_models_results.data.str.lower().unique()
    all_datasets_features = pd.DataFrame()
    for dataset in datasets:
        print(dataset)
        features = pd.read_csv(f'../results/features/test_{dataset}_features.csv')
        x = features.eb.values
        features.eb = np.where(x >= 1.0, x*0.01, x)
        features.eblc = features.eblc.str.upper()
        features.rename({'eblc': 'compression'}, axis=1, inplace=True)
        features = get_feature_diff_percentage(features) if 'solar' not in dataset else get_features_diff_percentage_solar(features)
        features.set_index(['compression', 'eb'], inplace=True)
        model_results = all_models_results[all_models_results.data == dataset.upper()]
        model_results = model_results[model_results.te_metric == 'nrmse']
        model_results = model_results[['eb', 'te', 'TFE', 'compression']]
        joined_df = features.merge(model_results.reset_index(drop=True), on=['compression', 'eb']).drop_duplicates()
        joined_df['data'] = dataset
        all_datasets_features = pd.concat([all_datasets_features, joined_df])

    all_datasets_features.to_csv('../results/features/all_datasets_features_diff_percentage.csv', index=False)
    return all_datasets_features.corr('spearman')


def compute_features_tfe_corr():
    all_models_results = pd.read_csv('../results/tfe/models_results.csv')
    datasets = all_models_results.data.str.lower().unique()
    all_datasets_features = pd.DataFrame()
    for dataset in datasets:
        print(dataset)
        features = pd.read_csv(f'../results/features/test_{dataset}_features.csv')
        x = features.eb.values
        features.eb = np.where(x >= 1.0, x*0.01, x)
        features.eblc = features.eblc.str.upper()
        features.rename({'eblc': 'compression'}, axis=1, inplace=True)
        features = get_feature_diff(features) if 'solar' not in dataset else get_features_diff_solar(features)
        features.set_index(['compression', 'eb'], inplace=True)
        model_results = all_models_results[all_models_results.data == dataset.upper()]
        model_results = model_results[model_results.te_metric == 'nrmse']
        model_results = model_results[['eb',  'te', 'TFE', 'compression']]
        joined_df = features.merge(model_results.reset_index(drop=True), on=['compression', 'eb']).drop_duplicates()
        joined_df['data'] = dataset
        all_datasets_features = pd.concat([all_datasets_features, joined_df])

    all_datasets_features.to_csv('../results/features/all_datasets_features_diff.csv', index=False)
    return all_datasets_features.corr('spearman')



In [6]:
compute_features_tfe_corr()

ettm2
ettm1
wind
solar
weather
aus


,eb,mean,var,max_kl_shift,time_kl_shift,max_level_shift,time_level_shift,max_var_shift,time_var_shift,x_acf1,...,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,peak,trough,te,TFE
eb,1.000000,-0.359034,-0.243797,0.601491,-0.092623,0.111844,0.012655,0.206705,0.096136,-0.198447,...,0.396365,-0.025559,0.179940,-0.123872,-0.117021,-0.492407,0.147704,0.058067,0.755653,0.660665
mean,-0.359034,1.000000,0.114055,-0.267071,-0.112175,0.074808,-0.111260,-0.182641,-0.088480,0.359082,...,-0.356634,-0.117547,-0.061338,0.355210,0.295988,0.238303,0.101138,0.002329,-0.443750,-0.448635
var,-0.243797,0.114055,1.000000,-0.482291,0.173313,-0.368105,0.159886,-0.291138,0.127246,-0.125419,...,-0.182867,-0.037653,-0.112462,0.209516,-0.062643,0.298927,0.095142,-0.357770,-0.544091,-0.471729
max_kl_shift,0.601491,-0.267071,-0.482291,1.000000,-0.056249,0.267627,-0.095061,0.322158,-0.080377,-0.227111,...,0.402394,0.091374,0.186993,-0.357020,-0.145459,-0.542245,-0.049538,0.195874,0.818096,0.766695
time_kl_shift,-0.092623,-0.112175,0.173313,-0.056249,1.000000,-0.164164,0.294844,-0.012175,0.226509,0.004957,...,-0.026763,0.087307,-0.067795,0.094092,0.082439,0.211817,-0.211141,0.022521,-0.113032,-0.106192
max_level_shift,0.111844,0.074808,-0.368105,0.267627,-0.164164,1.000000,-0.266466,0.400592,-0.121363,0.040686,...,0.233039,-0.213934,0.181885,-0.013215,0.050439,-0.285418,0.153042,0.022053,0.230332,0.231704
time_level_shift,0.012655,-0.111260,0.159886,-0.095061,0.294844,-0.266466,1.000000,-0.039405,0.355796,-0.049701,...,-0.016593,-0.022319,0.084425,0.044529,-0.037906,0.142394,-0.075600,0.031703,-0.094383,-0.085664
max_var_shift,0.206705,-0.182641,-0.291138,0.322158,-0.012175,0.400592,-0.039405,1.000000,0.037879,-0.094117,...,0.308884,0.147684,0.066448,-0.033382,-0.032224,-0.125600,0.001061,0.054160,0.259570,0.252370
time_var_shift,0.096136,-0.088480,0.127246,-0.080377,0.226509,-0.121363,0.355796,0.037879,1.000000,0.005830,...,-0.051957,-0.110798,0.160414,0.076564,0.053022,0.148567,-0.099034,-0.027053,-0.135018,-0.133507
x_acf1,-0.198447,0.359082,-0.125419,-0.227111,0.004957,0.040686,-0.049701,-0.094117,0.005830,1.000000,...,-0.214675,-0.098622,-0.005033,0.725630,0.795162,0.147952,-0.142212,0.355550,-0.175202,-0.305901


In [200]:
features_corr = compute_features_tfe_corr()
features_corr = features_corr[['TFE']].abs()
features_corr.drop(['eb', 'TFE'], inplace=True)
r = features_corr.sort_values(by='TFE', ascending=False)
r.index.name = 'ts_feature'
r.to_csv('../results/features/spearman_correlation.csv')

ettm2
ettm1
wind
solar
weather
aus


In [204]:
compute_features()

ettm2
ettm1
wind
solar
weather
aus


In [151]:
corr = compute_features_tfe_corr('test_ettm1', 'ETTM1')
# corr = corr[['TFE', 'te', 'compression ratio']]
features_of_interest = ['TFE', 'forecasting error']

for feature in features_of_interest:
    # for eblc in corr.compression.unique():
    print(f"Correlations with {feature}:")
    # Get correlations with the specific feature for each 'compression' group
    corr_feature = corr[[feature]] # .xs(key=feature, level=1, axis=1)[feature]

    # Drop the correlation of the feature with itself (it's always 1)
    corr_feature = corr_feature.drop(index=['eb', 'TFE', 'forecasting error', 'te', 'compression ratio'], errors='ignore')
    corr_feature[feature] = np.abs(corr_feature[feature].values)
    # Sort the correlations in descending order to get most correlated features first
    sorted_corr = corr_feature.sort_values(by=feature, ascending=False)

    # Display the top correlated features (e.g., top 5)
    print(sorted_corr.head(10))
    print("-----")

Correlations with TFE:
                        TFE
seasonal_strength  0.916137
var                0.869453
max_kl_shift       0.851600
flat_spots         0.798266
spike              0.701500
diff2x_pacf5       0.694868
nonlinearity       0.691763
diff1_acf1         0.671335
diff2_acf10        0.638921
alpha              0.638266
-----
Correlations with forecasting error:
                   forecasting error
seasonal_strength           0.735805
var                         0.731819
max_kl_shift                0.702418
flat_spots                  0.687810
nonlinearity                0.528525
spike                       0.504212
diff2x_pacf5                0.486277
mean                        0.474727
diff1_acf1                  0.467007
alpha                       0.460499
-----


In [147]:
corr_feature

,TFE
mean,-0.929065
var,-0.933789
max_kl_shift,0.759507
time_kl_shift,-0.229471
max_level_shift,0.165456
time_level_shift,-0.430137
max_var_shift,0.596724
time_var_shift,-0.137290
x_acf1,-0.468551
x_acf10,-0.444785


In [122]:
corr[[feature]].sort_values(by='TFE', ascending=False)

TFE
compression                                                  
PMC         TFE                                      1.000000
SZ          TFE                                      1.000000
SWING       TFE                                      1.000000
PMC         max_kl_shift                             0.986813
SWING       CO_FirstMin_ac                           0.984505
...                                                       ...
            nperiods                                      NaN
            seasonal_period                               NaN
SZ          IN_AutoMutualInfoStats_40_gaussian_fmmi       NaN
            nperiods                                      NaN
            seasonal_period                               NaN

[204 rows x 1 columns]

In [113]:
aus_results = compute_rho_tfe_te_cr_aus()
aus_results.groupby('eblc').corr(method='spearman')

dlinear_results.csv
informer_results.csv
nbeats_results.csv
transformer_results.csv


error       tfe
eblc                              
baseline error       NaN       NaN
         tfe         NaN       NaN
pmc      error  1.000000  0.970638
         tfe    0.970638  1.000000
swing    error  1.000000  0.980232
         tfe    0.980232  1.000000
sz       error  1.000000  0.973927
         tfe    0.973927  1.000000

In [44]:
compute_rho_eb_te_cr('aus')
compute_rho_tfe_te_cr('WIND')
compute_rho_tfe_te_cr('SOLAR')

eb  compression ratio       TFE        te
compression                                                                   
PMC         eb                 1.000000           1.000000  0.902292  1.000000
            compression ratio  1.000000           1.000000  0.902292  1.000000
            TFE                0.902292           0.902292  1.000000  0.902292
            te                 1.000000           1.000000  0.902292  1.000000
SWING       eb                 1.000000           1.000000  0.929307  1.000000
            compression ratio  1.000000           1.000000  0.929307  1.000000
            TFE                0.929307           0.929307  1.000000  0.929307
            te                 1.000000           1.000000  0.929307  1.000000
SZ          eb                 1.000000           1.000000  0.996319  0.995604
            compression ratio  1.000000           1.000000  0.996319  0.995604
            TFE                0.996319           0.996319  1.000000  0.991933
            te                 0.995604           0.995604  0.991933  1.000000

,compression ratio_TFE,compression ratio_te,TFE_te,eblc
correlation,9.022916e-01,1.000000e+00,9.022916e-01,PMC
pvalue,1.515094e-26,0.000000e+00,1.515094e-26,PMC
correlation,9.293069e-01,1.000000e+00,9.293069e-01,SWING
pvalue,3.957550e-31,0.000000e+00,3.957550e-31,SWING
correlation,9.963187e-01,9.956044e-01,9.919331e-01,SZ
pvalue,2.738345e-74,1.124893e-71,9.791567e-63,SZ


In [50]:
compute_rho_tfe_te_cr_pweather()

eb       TFE        te        cr
compression                                            
PMC         eb   1.000000  0.954628  1.000000  1.000000
            TFE  0.954628  1.000000  0.954628  0.954628
            te   1.000000  0.954628  1.000000  1.000000
            cr   1.000000  0.954628  1.000000  1.000000
SWING       eb   1.000000  0.981810  1.000000  1.000000
            TFE  0.981810  1.000000  0.981810  0.981810
            te   1.000000  0.981810  1.000000  1.000000
            cr   1.000000  0.981810  1.000000  1.000000
SZ          eb   1.000000  0.973918  1.000000  0.994505
            TFE  0.973918  1.000000  0.973918  0.969973
            te   1.000000  0.973918  1.000000  0.994505
            cr   0.994505  0.969973  0.994505  1.000000

,compression ratio_TFE,compression ratio_te,TFE_te,eblc
correlation,5.753655e-01,1.0,5.753655e-01,PMC
pvalue,3.507506e-06,0.0,3.507506e-06,PMC
correlation,9.802320e-01,1.0,9.802320e-01,SWING
pvalue,1.107683e-39,0.0,1.107683e-39,SWING
correlation,9.602217e-01,1.0,9.602217e-01,SZ
pvalue,1.359577e-31,0.0,1.359577e-31,SZ


In [19]:
compute_rho_tfe_te_cr('SOLAR')

eb  compression ratio       TFE        te
compression                                                                   
PMC         eb                 1.000000           1.000000  0.575365  1.000000
            compression ratio  1.000000           1.000000  0.575365  1.000000
            TFE                0.575365           0.575365  1.000000  0.575365
            te                 1.000000           1.000000  0.575365  1.000000
SWING       eb                 1.000000           1.000000  0.980232  1.000000
            compression ratio  1.000000           1.000000  0.980232  1.000000
            TFE                0.980232           0.980232  1.000000  0.980232
            te                 1.000000           1.000000  0.980232  1.000000
SZ          eb                 1.000000           1.000000  0.960222  1.000000
            compression ratio  1.000000           1.000000  0.960222  1.000000
            TFE                0.960222           0.960222  1.000000  0.960222
            te                 1.000000           1.000000  0.960222  1.000000

In [66]:
import statsmodels.formula.api as smf
for comp in ['pmc', 'swing', 'sz']:
    model = smf.ols(formula='cr ~ te', data=joined[(joined['compression'] == comp)]).fit()
    display(model.summary())

pmc


D:\Anaconda\envs\dart\lib\site-packages\scipy\stats\stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     cr   R-squared:                       0.336
Model:                            OLS   Adj. R-squared:                  0.275
Method:                 Least Squares   F-statistic:                     5.559
Date:                Tue, 12 Sep 2023   Prob (F-statistic):             0.0380
Time:                        18:11:54   Log-Likelihood:                -115.76
No. Observations:                  13   AIC:                             235.5
Df Residuals:                      11   BIC:                             236.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -968.9107    948.828     -1.021      0.329   -3057.268    1119.446
te          2.652e+04   1.12e+04      2.358      0.038    1763.935    5.13e+04
==============================================================================
Omnibus:                       13.955   Durbin-Watson:                   0.744
Prob(Omnibus):                  0.001   Jarque-Bera (JB):                9.446
Skew:                           1.557   Prob(JB):                      0.00889
Kurtosis:                       5.781   Cond. No.                         21.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

swing


D:\Anaconda\envs\dart\lib\site-packages\scipy\stats\stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     cr   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.559
Method:                 Least Squares   F-statistic:                     16.19
Date:                Tue, 12 Sep 2023   Prob (F-statistic):            0.00200
Time:                        18:11:54   Log-Likelihood:                -104.87
No. Observations:                  13   AIC:                             213.7
Df Residuals:                      11   BIC:                             214.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -620.9022    378.243     -1.642      0.129   -1453.410     211.605
te          1.184e+04   2943.105      4.024      0.002    5365.485    1.83e+04
==============================================================================
Omnibus:                        4.894   Durbin-Watson:                   0.781
Prob(Omnibus):                  0.087   Jarque-Bera (JB):                1.939
Skew:                           0.803   Prob(JB):                        0.379
Kurtosis:                       3.999   Cond. No.                         12.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

sz


D:\Anaconda\envs\dart\lib\site-packages\scipy\stats\stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     cr   R-squared:                       0.492
Model:                            OLS   Adj. R-squared:                  0.446
Method:                 Least Squares   F-statistic:                     10.66
Date:                Tue, 12 Sep 2023   Prob (F-statistic):            0.00753
Time:                        18:11:54   Log-Likelihood:                -69.281
No. Observations:                  13   AIC:                             142.6
Df Residuals:                      11   BIC:                             143.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     34.4219     20.900      1.647      0.128     -11.579      80.423
te           369.5957    113.191      3.265      0.008     120.464     618.728
==============================================================================
Omnibus:                       15.387   Durbin-Watson:                   2.568
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               12.916
Skew:                           1.413   Prob(JB):                      0.00157
Kurtosis:                       6.982   Cond. No.                         7.65
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""